In [11]:
import numpy as np
import cv2
import json
from scipy.optimize import least_squares

In [12]:
calib_path = "./data/calibration_data.json"

with open(calib_path, "r") as f:
    calib_data = json.load(f)

mtx = np.array(calib_data["mtx"])
dist = np.array(calib_data["dist"])
rvecs = np.array(calib_data["rvecs"])
tvecs = np.array(calib_data["tvecs"])

In [13]:
coord_path = "./data/coordinates_data.json"

with open(coord_path, "r") as f:
    coord_data = json.load(f)

objpoints = np.array(coord_data["objpoints"])
imgpoints = np.array(coord_data["imgpoints"])

In [14]:
def reprojection_error(params, objpoints, imgpoints, mtx, dist):
    n = len(objpoints)
    rvecs, tvecs = params[:n*3].reshape((n, 3)), params[n*3:].reshape((n, 3))

    # total_error = 0
    # for i in range(n):
    #     imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    #     error = np.sum((imgpoints[i] - imgpoints2.reshape(-1, 2))**2)
    #     total_error += error

    mean_error = 0
    for i in range(len(objpoints)):
        imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
        error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
        mean_error += error
    total_error = mean_error / len(objpoints)

    return total_error

In [16]:
initial_params = np.hstack((rvecs.ravel(), tvecs.ravel()))
result = least_squares(reprojection_error, initial_params, args=(objpoints, imgpoints, mtx, dist), verbose=2)

optimized_params = result.x
print(f"reprojection error: {result.cost:.3f}")
rvecs, tvecs = optimized_params[:len(objpoints)*3].reshape((len(objpoints), 3)), optimized_params[len(objpoints)*3:].reshape((len(objpoints), 3))
# print("rvecs:", rvecs)
# print("tvecs:", tvecs)

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         2.2401e-01                                    1.58e-05    
       1             16         2.2401e-01      0.00e+00       0.00e+00       1.58e-05    
`xtol` termination condition is satisfied.
Function evaluations 16, initial cost 2.2401e-01, final cost 2.2401e-01, first-order optimality 1.58e-05.
reprojection error: 0.224
